In [ ]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [ ]:
%%sql
sqlite:///financial_data_analysis.db

In [ ]:
import sqlite3
from sqlite3 import Error
import csv
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn

In [ ]:
def create_table(conn, create_table_sql, drop_table_name=None):

    if drop_table_name: # You can optionally pass drop_table_name to drop the table.
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Error as e:
            print(e)

    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [ ]:
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)
    rows = cur.fetchall()
    return rows

In [ ]:
normalized_database_filename="financial_data_analysis.db"
data_filename="Financial_data.csv"

Check total number of rows before removing empty rows.

In [ ]:
import pandas as pd

# Replace 'path/to/your/csv_file.csv' with the actual path to your CSV file
csv_file_path = '/content/Financial_data.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Get the number of rows
num_rows = len(df)

print(f"The number of rows in the CSV file is: {num_rows}")

<ipython-input-20-0879e470b36d>:7: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)


The number of rows in the CSV file is: 883752


Borrower and Project Name has empty values, so remove those rows

In [ ]:
# Remove rows with empty values in a specific column
column_name = 'Borrower'
column_name2 = 'Project Name'
df = df.dropna(subset=[column_name])
df = df.dropna(subset=[column_name2])
df = df.sample(frac=0.7, random_state=1)
# Save the modified DataFrame back to a CSV file if needed
df.to_csv('Financial_data.csv', index=False)



Check total number of rows after removing empty rows.

In [ ]:
import pandas as pd

# Replace 'path/to/your/csv_file.csv' with the actual path to your CSV file
csv_file_path = '/content/Financial_data.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Get the number of rows
num_rows = len(df)

print(f"The number of rows in the CSV file is: {num_rows}")

<ipython-input-59-0879e470b36d>:7: DtypeWarning: Columns (17,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)


The number of rows in the CSV file is: 1262503


Check if all data are present

In [ ]:
print(df.head())

            End of Period Credit Number                       Region  \
0  06/30/2019 12:00:00 AM      IDA51030                       AFRICA   
1  07/31/2017 12:00:00 AM      IDA44180      EUROPE AND CENTRAL ASIA   
2  05/31/2015 12:00:00 AM      IDA3411A                       AFRICA   
3  06/30/2015 12:00:00 AM      IDAH0210                       AFRICA   
4  06/30/2023 12:00:00 AM      IDA64850  EASTERN AND SOUTHERN AFRICA   

  Country Code       Country                                  Borrower  \
0           KE         Kenya  National Treasury  (Ministry of Finance)   
1           AZ    Azerbaijan                       MINISTRY OF FINANCE   
2           UG        Uganda  Ministry of Finance, Planning & Eco. Dev   
3           SL  Sierra Leone  Ministry Of Finance and Economic Develop   
4           RW        Rwanda   Ministry of Finance & Economic Planning   

     Credit Status  Service Charge Rate Currency of Commitment Project ID  \
0       Disbursing                 0.75      

Create table 1 - 'Countries'

In [ ]:
def table1_Countries(data_filename, normalized_database_filename):
    country_details=[]
    country_code_to_id = {}
    header=None
    with open(data_filename, 'r') as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)
        for line in csv_reader:
          region = line[2]
          country_code = line[3]
          country = line[4]
          if (region,country_code,country) not in country_details:
            country_details.append((region,country_code,country))

    conn_norm = create_connection(normalized_database_filename)
    create_table_1 = """
    CREATE TABLE [Countries] (
        [Country_ID] INTEGER PRIMARY KEY AUTOINCREMENT,
        [Region] TEXT NOT NULL,
        [Country_Code] TEXT NOT NULL,
        [Country] TEXT NOT NULL
    );
    """

    create_table(conn_norm, create_table_1, drop_table_name='Countries')

    sql_statement = "INSERT INTO Countries(Region, Country_Code, Country) VALUES(?, ?, ?)"

    cur = conn_norm.cursor()
    cur.executemany(sql_statement, country_details)

    # Fetch inserted data to create the mapping of Region to Country_ID
    cur.execute("SELECT Region, Country, Country_ID FROM Countries")
    for row in cur.fetchall():
        country_code_to_id[row[0],row[1]] = row[2]

    #print(country_code_to_id)

    conn_norm.commit()
    conn_norm.close()

    return country_code_to_id

country_code_to_id = table1_Countries(data_filename, normalized_database_filename)

Check for table 1

In [ ]:
print(country_code_to_id)

{('AFRICA', 'Kenya'): 1, ('EUROPE AND CENTRAL ASIA', 'Azerbaijan'): 2, ('AFRICA', 'Uganda'): 3, ('AFRICA', 'Sierra Leone'): 4, ('EASTERN AND SOUTHERN AFRICA', 'Rwanda'): 5, ('SOUTH ASIA', 'Pakistan'): 6, ('AFRICA', 'Mauritania'): 7, ('LATIN AMERICA AND CARIBBEAN', 'Nicaragua'): 8, ('EAST ASIA AND PACIFIC', "Lao People's Democratic Republic"): 9, ('EAST ASIA AND PACIFIC', 'Indonesia'): 10, ('SOUTH ASIA', 'India'): 11, ('AFRICA', 'Tanzania'): 12, ('EAST ASIA AND PACIFIC', 'China'): 13, ('LATIN AMERICA AND CARIBBEAN', 'Bolivia'): 14, ('EAST ASIA AND PACIFIC', 'Cambodia'): 15, ('MIDDLE EAST AND NORTH AFRICA', 'Djibouti'): 16, ('AFRICA', 'Ethiopia'): 17, ('EAST ASIA AND PACIFIC', 'Viet Nam'): 18, ('SOUTH ASIA', 'Afghanistan'): 19, ('MIDDLE EAST AND NORTH AFRICA', 'Yemen, Republic of'): 20, ('AFRICA', 'Senegal'): 21, ('Western and Central Africa', 'Burkina Faso'): 22, ('EUROPE AND CENTRAL ASIA', 'Kyrgyz Republic'): 23, ('AFRICA', 'Cameroon'): 24, ('EAST ASIA AND PACIFIC', 'Vietnam'): 25, ('A

Create table 2- 'Credit_Status_Details'

In [ ]:
def table2_Credit_Status_Details(data_filename, normalized_database_filename):
    credit_status_details=[]
    credit_status_dict={}
    header=None
    with open(data_filename, 'r',newline='') as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)
        for line in csv_reader:
          credit_status = line[6]
          credit_status_details.append((credit_status,))


    credit_status_details=list(set(credit_status_details))
    print(credit_status_details)
    conn_norm = create_connection(normalized_database_filename)
    create_table_2 = """
    CREATE TABLE [Credit_Status_Details] (
        [Credit_Status_ID] INTEGER PRIMARY KEY AUTOINCREMENT,
        [Credit_Status] TEXT NOT NULL
    );
    """

    create_table(conn_norm, create_table_2, drop_table_name='Credit_Status_Details')

    sql_statement = "INSERT INTO Credit_Status_Details(Credit_Status) VALUES(?)"

    cur = conn_norm.cursor()
    cur.executemany(sql_statement, credit_status_details)

    # Create a dictionary with credit status information
    cur.execute("SELECT Credit_Status_ID, Credit_Status FROM Credit_Status_Details")
    for row in cur.fetchall():
        print(row)
        credit_status_dict[row[1]] = row[0]

    conn_norm.commit()
    conn_norm.close()

    return credit_status_dict

credit_status_dict=table2_Credit_Status_Details(data_filename, normalized_database_filename)


[('Disbursing',), ('Signed',), ('Disbursing&Repaying',), ('Fully Repaid',), ('Terminated',), ('Cancelled',), ('Approved',), ('Negotiated',), ('Repaying',), ('Repaid',), ('Fully Disbursed',), ('Fully Cancelled',), ('Effective',), ('Disbursed',)]
(1, 'Disbursing')
(2, 'Signed')
(3, 'Disbursing&Repaying')
(4, 'Fully Repaid')
(5, 'Terminated')
(6, 'Cancelled')
(7, 'Approved')
(8, 'Negotiated')
(9, 'Repaying')
(10, 'Repaid')
(11, 'Fully Disbursed')
(12, 'Fully Cancelled')
(13, 'Effective')
(14, 'Disbursed')


Check for table 2

In [ ]:
print(credit_status_dict)

{'Disbursing': 1, 'Signed': 2, 'Disbursing&Repaying': 3, 'Fully Repaid': 4, 'Terminated': 5, 'Cancelled': 6, 'Approved': 7, 'Negotiated': 8, 'Repaying': 9, 'Repaid': 10, 'Fully Disbursed': 11, 'Fully Cancelled': 12, 'Effective': 13, 'Disbursed': 14}


Create table 3- 'Project_Details'

In [ ]:
def table3_Projects(data_filename, normalized_database_filename):
    credit_status_dict=table2_Credit_Status_Details(data_filename, normalized_database_filename)
    project_name_to_detail_id = {}
    project_details=[]

    project_name_list=[]
    header=None
    with open(data_filename, 'r') as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)
        for line in csv_reader:
          project_id = line[9]
          project_name = line[10]
          credit_status = line[6]
          country = line[4]
          region = line[2]

          # Capture Credit_Status_ID from the dictionary to store credit status information
          country_id = country_code_to_id.get((region,country))
          credit_status_id = credit_status_dict.get(credit_status)

          project_name_list.append(project_name)
          project_details.append((project_id,project_name,country_id,credit_status_id))

    project_details=list(set(project_details))
    print(len(set(project_name_list)))

    conn_norm = create_connection(normalized_database_filename)
    create_table_3 = """
    CREATE TABLE [Project_Details] (
        [Project_Detail_ID] INTEGER PRIMARY KEY AUTOINCREMENT,
        [Project_ID] VARCHAR,
        [Project_Name] TEXT NOT NULL,
        [Country_ID] INTEGER,
        [Credit_Status_ID] INTEGER,
        FOREIGN KEY (Country_ID) REFERENCES Countries(Country_ID),
        FOREIGN KEY (Credit_Status_ID) REFERENCES Credit_Status_Details(Credit_Status_ID)
    );
    """

    create_table(conn_norm, create_table_3, drop_table_name='Project_Details')

    sql_statement = "INSERT INTO Project_Details(Project_ID, Project_Name, Country_ID, Credit_Status_ID) VALUES(?, ?, ?, ?)"

    cur = conn_norm.cursor()
    cur.executemany(sql_statement, project_details)

    # Fetch project name and associated id to create a dictionary
    cur.execute("SELECT Project_ID, Project_Name, Country_ID, Project_Detail_ID FROM Project_Details")
    for row in cur.fetchall():
        project_name_to_detail_id[row[0],row[1],row[2]] = row[3]

    conn_norm.commit()
    conn_norm.close()

    return project_name_to_detail_id

project_name_to_detail_id=table3_Projects(data_filename, normalized_database_filename)

[('Disbursing',), ('Signed',), ('Disbursing&Repaying',), ('Fully Repaid',), ('Terminated',), ('Cancelled',), ('Approved',), ('Negotiated',), ('Repaying',), ('Repaid',), ('Fully Disbursed',), ('Fully Cancelled',), ('Effective',), ('Disbursed',)]
(1, 'Disbursing')
(2, 'Signed')
(3, 'Disbursing&Repaying')
(4, 'Fully Repaid')
(5, 'Terminated')
(6, 'Cancelled')
(7, 'Approved')
(8, 'Negotiated')
(9, 'Repaying')
(10, 'Repaid')
(11, 'Fully Disbursed')
(12, 'Fully Cancelled')
(13, 'Effective')
(14, 'Disbursed')
8094


Check for table 3

In [ ]:
print(project_name_to_detail_id)

{('P090656', 'ECSEE APL2 (ALBANIA)', 41): 37811, ('P159040', 'REDISSE 2', 243): 2, ('P128012', 'BD Ghorashal Unit 4 Repowering Proj', 29): 18513, ('P117229', 'KM DPL', 261): 4, ('P164283', 'Support to MSME Growth and Access to Fin', 26): 5, ('P155480', 'Chad-DPO Fiscal Consolidation Program', 126): 21130, ('P122157', 'RW-3rd Community Living Standards Grant', 164): 19851, ('P001224', 'EDUCATION I', 1): 8459, ('P082916', 'PUB FIN MGMT TA', 63): 33360, ('P125574', 'LR-Road Asset Management FY11)', 99): 19856, ('P106172', 'NG-Electricity and Gas Improvement', 108): 19193, ('P164654', 'KSEIP', 1): 18523, ('P117764', 'BJ-Decentralized Community Driven Servic', 103): 31173, ('P175820', 'RELI Project', 29): 30515, ('P145054', 'CASA-1000 Project', 6): 31178, ('P006153', 'AGRO EXPORT DEV PROG', 14): 22660, ('P161490', 'Zambia Integrated Forest Landscape (GEF)', 161): 19204, ('P078990', 'NI - EDUCATION', 8): 27710, ('P002972', 'UG EDUC SECTOR ADJ CRED', 252): 28344, ('P004834', 'IRRIGATION REHAB

Create table 4- 'Credit_Transactions'

In [ ]:
def table4_credit_transactions(data_filename, normalized_database_filename):
    credit_transactions = []
    header=None
    with open(data_filename, 'r') as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)
        for line in csv_reader:
          repaid_to_ida=0 #26 Feb 2024 12:59 pm - 2 rows with empty values
          disbursed_amount=0 #26 Feb 2024 12:59 pm - 2 rows with empty values
          original_principal_amount  = line[11]
          disbursed_amount = line[14]
          repaid_to_ida = line[15]
          credit_status = line[6]

          # To get project ID
          project_id = line[9]
          project_name = line[10]
          credit_status = line[6]
          country = line[4]
          region = line[2]

          # Capture Credit_Status_ID from the dictionary to store credit status information
          country_id = country_code_to_id.get((region,country))
          credit_status_id = credit_status_dict.get(credit_status)
          project_detail_id=project_name_to_detail_id[(project_id,project_name,country_id)]

          credit_transactions.append((project_detail_id,original_principal_amount, disbursed_amount, repaid_to_ida, credit_status_id))


    conn_norm = create_connection(normalized_database_filename)
    create_table_4 = """
    CREATE TABLE [Credit_Transactions] (
        [Financial_ID] INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        [Project_Detail_ID] INTEGER,
        [Credit_Status_ID] INTEGER,
        [Original_Principal_Amount] FLOAT NOT NULL,
        [Disbursed_Amount] FLOAT NOT NULL,
        [Repaid_to_IDA] FLOAT NOT NULL,
        FOREIGN KEY (Credit_Status_ID) REFERENCES Credit_Status_Details(Credit_Status_ID),
        FOREIGN KEY (Project_Detail_ID) REFERENCES Project_Details(Project_Detail_ID)
    );
    """


    create_table(conn_norm, create_table_4, drop_table_name = 'Credit_Transactions')

    sql_statement = "INSERT INTO Credit_Transactions(project_detail_id,Original_Principal_Amount, Disbursed_Amount, Repaid_to_IDA, Credit_Status_ID) VALUES(?, ?, ?, ?, ?)"

    cur = conn_norm.cursor()
    cur.executemany(sql_statement, credit_transactions)

    conn_norm.commit()
    conn_norm.close()

table4_credit_transactions(data_filename, normalized_database_filename)

Create table 5- 'Repayment_Schedule'

In [ ]:
import pandas as pd

df = pd.read_csv('/content/Financial_data.csv')
print(df.head())

def map_country_region_id(row, mapping_dict):
    # print(row)
    return mapping_dict.get((row['Region'], row['Country']))

df['country_region_id'] = df.apply(map_country_region_id, mapping_dict=country_code_to_id, axis=1)

sum_due_to_ida_by_country_region = df.groupby('country_region_id')['Due to IDA (US$)'].sum()
print(sum_due_to_ida_by_country_region)

<ipython-input-29-526f0a73024b>:3: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/Financial_data.csv')


            End of Period Credit Number                       Region  \
0  06/30/2019 12:00:00 AM      IDA51030                       AFRICA   
1  07/31/2017 12:00:00 AM      IDA44180      EUROPE AND CENTRAL ASIA   
2  05/31/2015 12:00:00 AM      IDA3411A                       AFRICA   
3  06/30/2015 12:00:00 AM      IDAH0210                       AFRICA   
4  06/30/2023 12:00:00 AM      IDA64850  EASTERN AND SOUTHERN AFRICA   

  Country Code       Country                                  Borrower  \
0           KE         Kenya  National Treasury  (Ministry of Finance)   
1           AZ    Azerbaijan                       MINISTRY OF FINANCE   
2           UG        Uganda  Ministry of Finance, Planning & Eco. Dev   
3           SL  Sierra Leone  Ministry Of Finance and Economic Develop   
4           RW        Rwanda   Ministry of Finance & Economic Planning   

     Credit Status  Service Charge Rate Currency of Commitment Project ID  \
0       Disbursing                 0.75      

In [ ]:
create_table_5 = """CREATE TABLE IF NOT EXISTS Repayment_Schedule (
    Repayment_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Country_ID INTEGER NOT NULL,
    Country TEXT NOT NULL,
    Region TEXT NOT NULL,
    Sum_due_to_IDA DECIMAL(10, 2),
    FOREIGN KEY (Country_ID) REFERENCES Countries(Country_ID)
);
"""
conn = sqlite3.connect('financial_data_analysis.db')
cursor = conn.cursor()
create_table(conn, create_table_5, drop_table_name = 'Repayment_Schedule')

In [ ]:
# Insert the data into CountryRegionSummary table
for (region, country), country_region_id in country_code_to_id.items():
    sum_due_to_ida = sum_due_to_ida_by_country_region.get(country_region_id, 0)
    cursor.execute('INSERT INTO Repayment_Schedule (Country_ID, Country, Region, Sum_due_to_IDA) VALUES (?, ?, ?, ?)',
                   (country_region_id, country, region, sum_due_to_ida))

# Commit the transaction and close the connection
conn.commit()
conn.close()

Create table 6- 'Borrowers'

In [ ]:
def table5_borrowers(data_filename, normalized_database_filename):
    borrowers = []
    with open(data_filename, 'r') as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)
        for line in csv_reader:
          borrower_obligation=0 #26 Feb 2024 12:59 pm - 2 rows with empty values
          borrower = (line[15].replace(",", "").replace("'", "").strip() or 0)
          borrower_obligation  = line[18]
          country = line[4]
          region = line[2]

          # Capture Credit_Status_ID from the dictionary to store credit status information
          country_id = country_code_to_id.get((region,country))
          borrowers.append((borrower, borrower_obligation,country_id))

    conn_norm = create_connection(normalized_database_filename)
    create_table_5 = """
    CREATE TABLE [Borrowers] (
        [Borrower_ID] INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        [Borrower] VARCHAR NOT NULL,
        [Country_ID] INTEGER NOT NULL,
        [Borrowers_Obligation] FLOAT NOT NULL,
        FOREIGN KEY (Country_ID) REFERENCES Countries(Country_ID)
    );
    """


    create_table(conn_norm, create_table_5, drop_table_name = 'Borrowers')

    sql_statement = "INSERT INTO Borrowers(Borrower, Borrowers_Obligation,Country_ID) VALUES(?, ?, ?)"

    cur = conn_norm.cursor()
    cur.executemany(sql_statement, borrowers)

    conn_norm.commit()
    conn_norm.close()

table5_borrowers(data_filename, normalized_database_filename)